<a href="https://colab.research.google.com/github/ii200400/Tensorflow_Tutorial/blob/master/10%20-%20RNN/02_Autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

자연어 처리나 음성 처리 분야에 많이 사용되는 RNN 의 기본적인 사용법을 익히자!

4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성하는 프로그램을 만들었다.

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

TensorFlow 1.x selected.


## 데이터 정의

평소처럼 이미 있는 데이터 셋을 가져와서 하는 것이 아니라 임의로 데이터를 만들어서 훈련 및 테스트를 한다.

때문에 아주 제한적인 상황에서만 쓰일 수 있는 모델이 만들어지며 사용하는 데이터는 아래와 같다.

In [ ]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# one-hot 인코딩 및 디코딩을 사용 하기 위해 딕셔너리를 만든다.
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 다음 배열은 입력값과 출력값으로 아래와 같은 형식을 가진다.
# wor -> X, d -> Y
# woo -> X, d -> Y
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

### 데이터 인코딩

In [ ]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        # 여기서 생성하는 input_batch 와 target_batch 는
        # 알파벳 배열의 인덱스 번호이다.
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
        input = [num_dic[n] for n in seq[:-1]]

        # 3, 3, 15, 4, 3 ...
        target = num_dic[seq[-1]]

        # one-hot 인코딩을 한다.
        # input 이 [0, 1, 2] 라면
        # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
        # numpy-array[리스트] 와 같은 형식은 아래의 셀에 예시를 만들어 놓았다.
        input_batch.append(np.eye(dic_len)[input])

        # 지금까지 손실함수로 사용하던 softmax_cross_entropy_with_logits 함수는
        # label 값을 one-hot 인코딩으로 넘겨줘야 하지만,
        # 이 예제에서 사용할 손실 함수인 sparse_softmax_cross_entropy_with_logits 는
        # one-hot 인코딩을 사용하지 않으므로 index 로 바로 넘긴다.
        target_batch.append(target)

    return input_batch, target_batch

#### np[list] 예시

In [ ]:
# numpy array 옆에 리스트를 사용하면 해당 행을 뽑아와서 새로운 np를 만든다.
test_np = np.array([[1,2,3,4,5],[5,4,3,2,1]])
pick = [1,0,1]

print(test_np[pick])

[[5 4 3 2 1]
 [1 2 3 4 5]
 [5 4 3 2 1]]


## 모델 구성

### 옵션 설정

In [ ]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

# RNN 을 구성하는 시퀀스의 갯수 => 단어의 길이
n_step = 3

# 입력값과 출력값의 크기는
# 알파벳에 대한 one-hot 인코딩이므로 26개가 된다.
# 예) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
n_input = n_class = dic_len

### 변수 정의

In [ ]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])

# 비용함수는 인덱스 숫자를 그대로 사용하는 sparse_softmax_cross_entropy_with_logits 이기 때문에
# 입력값은 아래와 같은 인덱스 하나의 값의 1차원 배열이다.
# [3] [3] [15] [4] ...
# 기존처럼 one-hot 인코딩을 사용한다면 입력값의 형태는 [None, n_class] 이어야 한다.
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

### 신경망 구성

In [ ]:
# RNN 셀을 생성한다.
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 과적합 방지를 위한 Dropout 기법을 사용하고
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
# 여러개의 셀을 조합해서 사용하기 위해 셀을 추가로 생성한다.
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# 위에서 만든 셀들을 조합한 RNN 셀을 생성한다.
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

# tf.nn.dynamic_rnn 함수를 이용해 순환 신경망을 만든다.
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

# 최종 결과는 one-hot 인코딩 형식으로 하였다.
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### 비용 및 최적화

In [ ]:
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 모델 학습

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

InvalidArgumentError: ignored

### 결과 확인

In [ ]:
# 레이블값이 정수이므로 예측값도 정수로 출력하도록 한다.
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
# 정수인 입력값을 비교한다.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0


## 전체 코드

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

#########
# 데이터 정의
######

char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]

        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)

    return input_batch, target_batch

#########
# 옵션 설정
######

learning_rate = 0.01
n_hidden = 128
total_epoch = 30

n_step = 3
n_input = n_class = dic_len

#########
# 신경망 모델 구성
######

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#########
# 신경망 모델 학습
######

sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

#########
# 결과 확인
######

prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)

TensorFlow 1.x selected.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch: 0001 cost = 3.589193
Epoch: 0002 cost = 2.705246
Epoch: 0003 cost = 1.756665
Epoch: 0004 cost = 1.043695
Epoch: 0005 cost = 1.199448
Epoch: 0006 cost = 0.370280
Epoch: 0007 cost = 0.532801
Epoch: 0008 cost = 0.443687
Epoch: 0009 cost = 0.311985
Epoch: 0010 cost = 0.447316
Epoch: 0011 cost = 0.268215
Epoch: 0012 cost = 0.222872
Epoch: 0013 cost = 0.418366
Epoch: 0014 cost = 0.253673
Epoch: 0015 cost = 0.105682
Epo